## Google Analytics API Setup Guide

**Official Guide (v4)**

* https://developers.google.com/analytics/devguides/reporting/core/v4/quickstart/service-py

**Query Builder**

* https://ga-dev-tools.appspot.com/request-composer/

**Resources Consulted**

* http://www.ryanpraski.com/google-analytics-reporting-api-python-tutorial/
* http://serhiipuzyrov.com/2019/03/google-analytics-api-request-python
* https://www.themarketingtechnologist.co/getting-started-with-the-google-analytics-reporting-api-in-python/

## Initial Setup

Following the developers guide, you will need to create a new service account (and project if you do not have one) and obtain the related JSON containing your private keys to connect to the Google Analytics API.

The steps are summarized below, taken directly from the developers guide:

![Official Steps](https://github.com/kn-kn/python-guide/blob/master/Google%20API%20Guide/Official_Steps_List.PNG?raw=true)

---

**1. First, if you have not created a project, create a project**

![Create Project](https://github.com/kn-kn/python-guide/blob/master/Google%20API%20Guide/Create_Project.PNG?raw=true)

---

**2. Press the "+ Create Service" button to create a service. Set a service account name. The other sections are optional.**

![Press Create Service](https://github.com/kn-kn/python-guide/blob/master/Google%20API%20Guide/Press_Create_Service.PNG?raw=true)

---

![Service Settings](https://github.com/kn-kn/python-guide/blob/master/Google%20API%20Guide/Service_Settings.PNG?raw=true)

---

**3. Download the key file in JSON format. This file is extremely important. Save it in a location you will remember!**

![Create Key](https://github.com/kn-kn/python-guide/blob/master/Google%20API%20Guide/Create_Key.PNG?raw=true)

---

**4. Now you will need to give read & analyze access to this service account to the Google Analytics view that you want to use.**

**To do this, use the email given to you once you've finished the previous step (see image below). Official support documents for adding a new user to a Google view can be found in the following:**

https://support.google.com/analytics/answer/1009702

![Email](https://github.com/kn-kn/python-guide/blob/master/Google%20API%20Guide/Email.PNG?raw=true)

![GA Rights](https://github.com/kn-kn/python-guide/blob/master/Google%20API%20Guide/GA_Rights.PNG?raw=true)

**5. Install the Google API python package. Use a package manager like `pip` to install it**.

`pip install --upgrade google-api-python-client`

## Code Setup

In [ ]:
import pandas as pd
import numpy as np

The following cell contains code directly from the Google Analytics API Guide. Copy it and change the following two variables:

* **SCOPES**: We've set our scope to read only. Use the URL given below for our example.
* **KEY_FILE_LOCATION**: The previously downloaded JSON format. Point to its location.
* **VIEW_ID**: The view ID you gave permission to your service earlier.

In [ ]:
"""Hello Analytics Reporting API V4."""

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

# Ensure your variables are pointing to the right location!
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '<REPLACE_WITH_JSON_FILE>'
VIEW_ID = '<REPLACE_WITH_VIEW_ID>'

def initialize_analyticsreporting():
  """Initializes an Analytics Reporting API V4 service object.

  Returns:
    An authorized Analytics Reporting API V4 service object.
  """
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics

Using the Google Analytics Request Composer found here: https://ga-dev-tools.appspot.com/request-composer/

Build the query you wish to request from Google API. Place that query into the body variable below.

The second function in the cell below, `print_response()` contains code that will transform the output of `get_report()` which is a dictionary, into a Pandas Dataframe.

**Important Note**: The Google Analytics API will always a maximum of 100k rows per result. To set it as 100k, add the `pageSize` parameter found below. If you wish to query more than 100k rows, consider creating a loop that appends multiple calls of smaller date ranges.

In [ ]:
def get_report(analytics):
    return analytics.reports().batchGet(
        # Place the dictionary you get from the Google Query Builder into the body variable below
        body={
            "reportRequests": [
                {
                    "viewId": "<REPLACE_WITH_VIEW_ID>",
                    # Set the max rows to return. 100k is the Google Analytics API maximum it will return
                    "pageSize": 100000,
                    "filtersExpression": "ga:pagePath=yourwebsitehere.com",
                    "dateRanges": [{"startDate": "2019-06-01", "endDate": "2019-06-30"}],
                    "metrics": [{"expression": "ga:uniquePageviews", "alias": "UniquePageviews"}],
                    "dimensions": [{"name": "ga:pagePath"}, {"name": "ga:mobileDeviceInfo"}]
                }]
        }

    ).execute()

response = get_report(initialize_analyticsreporting())

# Transforms the dictionary response into a pandas dataframe
def print_response(response):
    list = []
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
        for row in rows:
            dict = {}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])
            for header, dimension in zip(dimensionHeaders, dimensions):
                dict[header] = dimension
            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    if ',' in value or '.' in value:
                        dict[metric.get('name')] = float(value)
                    else:
                        dict[metric.get('name')] = int(value)
            list.append(dict)
        df = pd.DataFrame(list)
        return df

# Place final result into pandas dataframe
df = print_response(response)

Your final result will be in the dataframe, `df`. You can now run your Python commands on `df`.